In [1]:
import pandas as pd


In [6]:
data = pd.read_csv('Task_1\images\metadata.csv')
data.head()

C:\Users\yannn\AppData\Local\Temp\ipykernel_29436\1000669844.py:1: DtypeWarning: Columns (8,11,13,14,15,17,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Task_1\images\metadata.csv')


,isic_id,attribution,copyright_license,acquisition_day,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,dermoscopic_type,diagnosis,...,mel_thick_mm,mel_type,mel_ulcer,melanocytic,nevus_type,patient_id,personal_hx_mm,pixels_x,pixels_y,sex
0,ISIC_3491559,Memorial Sloan Kettering Cancer Center,CC-0,NaN,45.0,upper extremity,benign,1.4,NaN,seborrheic keratosis,...,NaN,NaN,NaN,True,NaN,IP_0918668,False,3264,2448,female
1,ISIC_3622379,Memorial Sloan Kettering Cancer Center,CC-0,NaN,65.0,posterior torso,benign,8.8,contact polarized,nevus,...,NaN,NaN,NaN,True,NaN,IP_6028537,True,3264,2448,male
2,ISIC_2443008,Memorial Sloan Kettering Cancer Center,CC-0,NaN,45.0,upper extremity,malignant,7.7,non-contact polarized,squamous cell carcinoma,...,NaN,NaN,NaN,False,NaN,IP_2891227,False,3264,2448,male
3,ISIC_6892286,Memorial Sloan Kettering Cancer Center,CC-0,NaN,75.0,posterior torso,malignant,9.4,contact polarized,melanoma,...,0.3,NaN,False,True,NaN,IP_3056186,True,3264,2448,male
4,ISIC_1979534,Memorial Sloan Kettering Cancer Center,CC-0,NaN,80.0,head/neck,malignant,20.0,NaN,squamous cell carcinoma,...,NaN,NaN,NaN,False,NaN,IP_2084621,True,3264,2448,male


In [7]:
columns = data.columns.tolist()

In [9]:
print(columns)

['isic_id', 'attribution', 'copyright_license', 'acquisition_day', 'age_approx', 'anatom_site_general', 'benign_malignant', 'clin_size_long_diam_mm', 'dermoscopic_type', 'diagnosis', 'diagnosis_confirm_type', 'family_hx_mm', 'image_type', 'lesion_id', 'mel_class', 'mel_mitotic_index', 'mel_thick_mm', 'mel_type', 'mel_ulcer', 'melanocytic', 'nevus_type', 'patient_id', 'personal_hx_mm', 'pixels_x', 'pixels_y', 'sex']


In [65]:
df = data[['isic_id', 'diagnosis', 'benign_malignant']]

In [66]:
images = df.isic_id.tolist()


labels = df.diagnosis.tolist()

In [67]:
import torch
import lightning

In [68]:
from sklearn import model_selection, preprocessing

In [69]:
le = preprocessing.LabelEncoder()
encoded = le.fit_transform(labels)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'AIMP': 0, 'acrochordon': 1, 'actinic keratosis': 2, 'angiofibroma or fibrous papule': 3, 'angiokeratoma': 4, 'angioma': 5, 'atypical melanocytic proliferation': 6, 'atypical spitz tumor': 7, 'basal cell carcinoma': 8, 'cafe-au-lait macule': 9, 'clear cell acanthoma': 10, 'dermatofibroma': 11, 'lentigo NOS': 12, 'lentigo simplex': 13, 'lichenoid keratosis': 14, 'melanoma': 15, 'melanoma metastasis': 16, 'nan': 17, 'neurofibroma': 18, 'nevus': 19, 'other': 20, 'pigmented benign keratosis': 21, 'scar': 22, 'seborrheic keratosis': 23, 'solar lentigo': 24, 'squamous cell carcinoma': 25, 'vascular lesion': 26, 'verruca': 27}


In [70]:
trainx, testx, trainy, testy = model_selection.train_test_split(images, encoded, test_size=.3)

In [71]:
trainx, valx, trainy, valy = model_selection.train_test_split(trainx, trainy, train_size=.8)

In [74]:
def make_csv(images, labels, name):
    df = pd.DataFrame([[image, label] for image, label in zip(images, labels) 
                       ], columns=['images', 'labels'])
    df.to_csv(f'{name}.csv', index=False)

In [75]:
make_csv(trainx,trainy,'train')
make_csv(valx,valy,'val')
make_csv(testx,testy,'test')

In [79]:
!python imagenet.py --arch mobilenet_v3_small --accelerator gpu --auto_scale_batch_size --precision 16 --amp_backend apex --amp_level 'O3' --data-path Task_1/images/ --pretrained 

usage: imagenet.py [--logger [LOGGER]]
                   [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                   [--default_root_dir DEFAULT_ROOT_DIR]
                   [--gradient_clip_val GRADIENT_CLIP_VAL]
                   [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                   [--num_nodes NUM_NODES] [--num_processes NUM_PROCESSES]
                   [--devices DEVICES] [--gpus GPUS]
                   [--auto_select_gpus [AUTO_SELECT_GPUS]]
                   [--tpu_cores TPU_CORES] [--ipus IPUS]
                   [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                   [--overfit_batches OVERFIT_BATCHES]
                   [--track_grad_norm TRACK_GRAD_NORM]
                   [--check_val_every_n_epoch CHECK_VAL_EVERY_N_EPOCH]
                   [--fast_dev_run [FAST_DEV_RUN]]
                   [--accumulate_grad_batches ACCUMULATE_GRAD_BATCHES]
                   [--max_epochs MAX_EPOCHS] [--min_epochs MIN_EPOCHS]
                   [--ma